# AWS Announcements Assistant

### Evaluate 🔎 AWS Announcements and auto-generate a presentation. This sample will fetch 🎣 latest AWS annoucements, using Amazon Bedrock will summarize the details and extract technology names from the details and auto-generate a presentation for you."


In [22]:
!pip install bs4
!pip install requests 
!pip install langchain
!pip install python-pptx


[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [39]:
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import requests
import shutil
import time

from datetime import date
from datetime import timedelta
from datetime import datetime
from langchain import PromptTemplate

from pptx import Presentation
from pptx.util import Inches
from pptx.util import Cm, Pt

from datetime import datetime

import json
import boto3
import uuid

from botocore.exceptions import ClientError

In [40]:
#This will fetch a specific AWS annoucment, extract publishing date, title and body of annoucment

link = "https://aws.amazon.com/about-aws/whats-new/2023/10/aws-systems-manager-application-manager-sap-hana/"

r  = requests.get(link)

data = r.text

soup = BeautifulSoup(data,'html.parser')

final_text_title = ''
final_text_body = ''

annoucment_title = soup.find_all('div',{"class": "title-wrapper"})
annoucment_date = soup.find('span',{"class": "date"}).text

print(annoucment_date)


for lists in annoucment_title:
	element_checked = lists.find('a')
	if element_checked!=None:
		text_got = lists.find('a').text
		final_text_title = final_text_title + text_got
        
print("Title of the annoucment : ")
print(final_text_title)
print("\n")

nestedDiv_list = soup.find_all('div',{"class": "aws-text-box"})

for lists in nestedDiv_list:
	element_checked = lists.find('p')
	if element_checked!=None:
		text_got = lists.find('p').text
		final_text_body = final_text_body + text_got

print("Body of the annoucment : ")
print(final_text_body)
print("\n")

 Oct 11, 2023
Title of the annoucment : 
 AWS Systems Manager Application Manager now supports SAP HANA


Body of the annoucment : 
You can now use AWS Systems Manager Application Manager to perform operational activities with SAP HANA databases in addition to command line interfaces. AWS Systems Manager for SAP also now supports highly available SAP HANA deployments.AWS Systems Manager is a secure end-to-end management solution for resources on AWS. Application Manager, a capability of AWS Systems Manager, helps DevOps engineers investigate and remediate issues with their AWS resources in the context of their applications and clusters. AWS Systems Manager for SAP helps AWS services understand SAP specific nuances around landscape topology and discover the components of an SAP landscape, such the SAP NetWeaver application server, SAP HANA database, and their interdependencies. It provides automation capabilities to help customers manage and operate their SAP applications on AWS more ef

In [41]:
#bedrock_client = boto3.client('bedrock' , 'us-west-2', endpoint_url = 'https://bedrock.us-west-2.amazonaws.com')
bedrock_client = boto3.client(
    service_name='bedrock-runtime', 
    region_name='us-west-2'
)

In [45]:
# Interact with a large language model (LLM) to generate text 
# based on a prompt.
#
# Arguments:
#   prompt: The text prompt to provide to the LLM.
#   llm_type: The name of the LLM to use, either 'titan' or 'claude'. 
#
# Returns:
#   The text generated by the LLM in response to the prompt.
#   
# This function:
# 1. Prints the llm_type for debugging.
# 2. Formats the prompt into the JSON payload expected by each LLM API.
# 3. Specifies the parameters for text generation like max tokens, temp.
# 4. Calls the Bedrock client to invoke the LLM model API. 
# 5. Parses the response to extract the generated text.
# 6. Returns the generated text string.

def interactWithLLM(prompt, type):

    if type == "titan":
        print("**THE LLM TYPE IS -->" + type)
        print("prompt---->" + prompt)
        # Test for invoke model begins
        parameters = {
            "maxTokenCount": 512,
            "stopSequences": [],
            "temperature": 0,
            "topP": 0.9,
        }
        body = json.dumps({"inputText": prompt, "textGenerationConfig": parameters})
        modelId = "amazon.titan-tg1-large"  # "amazon.titan-tg1-large"
        accept = "application/json"
        contentType = "application/json"
        response = bedrock_client.invoke_model(
            body=body, modelId=modelId, accept=accept, contentType=contentType
        )
        response_body = json.loads(response.get("body").read())

        response_text_titan = response_body.get("results")[0].get("outputText")

        return response_text_titan

    elif type == "claude":
        print("**THE LLM TYPE IS -->" + type)
        body = json.dumps(
            {
                "prompt": prompt,
                "max_tokens_to_sample": 300,
                "temperature": 1,
                "top_k": 250,
                "top_p": 0.999,
                "stop_sequences": [],
            }
        )
        modelId = "anthropic.claude-v2"  # change this to use a different version from the model provider
        accept = "application/json"
        contentType = "application/json"
        print("prompt---->" + prompt)
        response = bedrock_client.invoke_model(
            body=body, modelId=modelId, accept=accept, contentType=contentType
        )
        response_body = json.loads(response.get("body").read())

        response_text_claude = response_body.get("completion")

        return response_text_claude
    
    elif type == 'jurassic':
        print("**THE LLM TYPE IS -->" + type)
        body = json.dumps({"prompt":prompt,"maxTokens":200,"temperature":0,"topP":1,"stopSequences":[],"countPenalty":{"scale":0},"presencePenalty":{"scale":0},"frequencyPenalty":{"scale":0}}) 
        modelId = 'ai21.j2-ultra' # change this to use a different version from the model provider
        accept = 'application/json'
        contentType = 'application/json'
        print("prompt---->" + prompt)
        response = bedrock_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
        response_body = json.loads(response.get('body').read())
        
        response_text_jurassic = response_body.get('completions')[0].get("data").get("text")
        
        return response_text_jurassic


In [46]:
prompt_takeaways_tech_claude = """
Human:  Here are the details of annoucements:
<annoucements>
{text}
</annoucements>
Give 2 key takeaways from the above annoucements and list technology names also. In the output takeaways under "Takeaways:" and list technology names under "Technology names:"
Assistant: 
	"""

In [47]:
#using Claude as an example

llm_type = "claude"

prompt_template_for_summary_generate = PromptTemplate.from_template(prompt_takeaways_tech_claude)
prompt_data_for_summary_generate = prompt_template_for_summary_generate.format(text=final_text_body)

response_text = interactWithLLM(prompt_data_for_summary_generate,llm_type)


print("Response from LLM is ready")

print("Response from LLM : ")
print(response_text)

**THE LLM TYPE IS -->claude
prompt---->
Human:  Here are the details of annoucements:
<annoucements>
You can now use AWS Systems Manager Application Manager to perform operational activities with SAP HANA databases in addition to command line interfaces. AWS Systems Manager for SAP also now supports highly available SAP HANA deployments.AWS Systems Manager is a secure end-to-end management solution for resources on AWS. Application Manager, a capability of AWS Systems Manager, helps DevOps engineers investigate and remediate issues with their AWS resources in the context of their applications and clusters. AWS Systems Manager for SAP helps AWS services understand SAP specific nuances around landscape topology and discover the components of an SAP landscape, such the SAP NetWeaver application server, SAP HANA database, and their interdependencies. It provides automation capabilities to help customers manage and operate their SAP applications on AWS more efficiently. This enables custome

In [48]:
#preparing to save prompt and its responses to dynamodb

dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('llm_track')

In [49]:
table_name = 'llm_track'

# Check if table exists
table = dynamodb.Table(table_name)
try:
    table.load()
except ClientError as err:
    if err.response['Error']['Code'] == 'ResourceNotFoundException':
        # Table does not exist, create it
        table = dynamodb.create_table(
                TableName='llm_track',
                KeySchema=[
                    {
                        'AttributeName': 'llm_type',
                        'KeyType': 'HASH' 
                    },
                    {
                        'AttributeName': 'insert_time',
                        'KeyType': 'RANGE'  
                    }
                ],
                AttributeDefinitions=[
                    {
                        'AttributeName': 'llm_type',
                        'AttributeType': 'S'
                    },
                    {
                        'AttributeName': 'insert_time',
                        'AttributeType': 'S'
                    }

                ],
                ProvisionedThroughput={
                    'ReadCapacityUnits': 10,
                    'WriteCapacityUnits': 10
                }
            )
        
        print("Table status:", table.table_status)
    else:
        raise
else:
    print("Table exists")

Table exists


In [50]:
#function to save prompt and its responses to dynamodb

def savetoDDB(type,prompt, response_llm,parsedDate,link,typeOfExtract):
	now = datetime.now()
	date_time = now.strftime("%m/%d/%Y, %H:%M:%S")

	response = table.put_item(
			Item={
				'llm_type': type,
				'insert_time': date_time,
				'prompt' : prompt,
				'response' : response_llm,
				'parsedDate' : parsedDate,
				'link' : link,
				'typeOfExtract' : typeOfExtract
			}
		)
	
	status_code = response['ResponseMetadata']['HTTPStatusCode']
	return status_code

In [51]:
savetoDDB(llm_type,prompt_data_for_summary_generate,response_text,annoucment_date,link,"summarize_and_tech")

200

In [52]:
#auto generates a presentation based on the output of Amazon Bedrock

# Create a new Presentation
prs=Presentation()

# Select the first slide layout as template 
lyt=prs.slide_layouts[0] # choosing a slide layout

# Add a new slide using the template layout
slide=prs.slides.add_slide(lyt) # adding a slide

# Get references to title and subtitle placeholders
title=slide.shapes.title # assigning a title

subtitle=slide.placeholders[1] # placeholder for subtitle

# Set title and subtitle text
title.text="AWS whats new" # title
subtitle.text="All AWS annoucments for " + annoucment_date # subtitle

# Create a second slide using a different layout template
Second_Layout = prs.slide_layouts[5]
second_slide = prs.slides.add_slide(Second_Layout)

# Set title text for second slide
second_slide.shapes.title.text = final_text_title
title_para = second_slide.shapes.title.text_frame.paragraphs[0]

# Set font size for title
title_para.font.size = Pt(18)

# Add textbox and set properties
textbox = second_slide.shapes.add_textbox(Inches(1), Inches(1.5),Inches(9), Inches(5))
textframe = textbox.text_frame
textframe.word_wrap = True 

# Add paragraph with response text
paragraph = textframe.add_paragraph()
paragraph.text = response_text + "\n\n" + link
paragraph.font.size = Pt(18)

# Save the presentation
prs.save("sample.pptx")

In [53]:
prompt_takeaways_tech_jurassic = """
Human:  Here are the details of annoucements:
{text}
Give 2 key takeaways from the above annoucements and list technology names also. In the output takeaways under "Takeaways:" and list technology names under "Technology names:"
	"""

In [54]:
#using Claude as an example

llm_type = "jurassic"

prompt_template_for_summary_generate = PromptTemplate.from_template(prompt_takeaways_tech_jurassic)
prompt_data_for_summary_generate = prompt_template_for_summary_generate.format(text=final_text_body)

response_text = interactWithLLM(prompt_data_for_summary_generate,llm_type)


print("Response from LLM is ready")

**THE LLM TYPE IS -->jurassic
prompt---->
Human:  Here are the details of annoucements:
You can now use AWS Systems Manager Application Manager to perform operational activities with SAP HANA databases in addition to command line interfaces. AWS Systems Manager for SAP also now supports highly available SAP HANA deployments.AWS Systems Manager is a secure end-to-end management solution for resources on AWS. Application Manager, a capability of AWS Systems Manager, helps DevOps engineers investigate and remediate issues with their AWS resources in the context of their applications and clusters. AWS Systems Manager for SAP helps AWS services understand SAP specific nuances around landscape topology and discover the components of an SAP landscape, such the SAP NetWeaver application server, SAP HANA database, and their interdependencies. It provides automation capabilities to help customers manage and operate their SAP applications on AWS more efficiently. This enables customers to capture

In [55]:
print("Response from LLM : ")
print(response_text)

Response from LLM : 
1. AWS Systems Manager Application Manager now supports operational activities with SAP HANA databases in addition to command line interfaces.
 2. AWS Systems Manager for SAP now supports highly available SAP HANA deployments.
Technology names:
AWS Systems Manager, AWS Systems Manager for SAP, SAP HANA
